<a href="https://colab.research.google.com/github/mohantyipsita/MassQL_bile_Acid_isomer_2025/blob/main/massql_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas massql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00


In [2]:
!python --version

Python 3.12.12


In [3]:
pip show massql

Name: massql
Version: 2025.12.10
Summary: Mass spectrometry query language python implementation
Home-page: https://github.com/mwang87/MassQueryLanguage
Author: Mingxun Wang
Author-email: mwang87@gmail.com
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: kaleido, lark-parser, pandas, plotly, psims, py_expression_eval, pyarrow, pydot, pymzml, pyteomics, pyyaml, tqdm
Required-by: 


In [4]:
from google.colab import files

uploaded = files.upload()  # choose your .mgf file in the popup
uploaded

Saving Attached standard file_ GCDCA_TCA_for_authors_to_reproduce_GitHub_error.mgf to Attached standard file_ GCDCA_TCA_for_authors_to_reproduce_GitHub_error.mgf


{'Attached standard file_ GCDCA_TCA_for_authors_to_reproduce_GitHub_error.mgf': b'BEGIN IONS\r\nSCANS=1\r\nTITLE=GCDCA [M+H]+ HCD 45V\r\nPEPMASS=450.3214\r\nCHARGE=1+\r\n50.8482 35.26\r\n53.4529 29.96\r\n55.0551 66.64\r\n62.8912 31.91\r\n64.9888 34.96\r\n67.055 132.49\r\n69.0706 45.55\r\n73.3333 36.17\r\n76.0401 999.0\r\n79.0549 70.78\r\n81.0706 331.41\r\n83.0862 81.64\r\n83.2548 34.32\r\n84.0004 31.07\r\n85.0655 296.75\r\n85.0683 46.95\r\n91.0547 65.16\r\n93.0705 259.7\r\n95.0861 366.05\r\n105.0704 247.11\r\n107.0861 364.48\r\n109.1017 283.95\r\n119.0859 303.38\r\n121.1016 426.24\r\n122.1046 37.69\r\n123.0808 45.5\r\n123.1172 94.83\r\n131.0856 127.54\r\n133.1015 359.32\r\n134.1048 41.22\r\n135.1173 293.63\r\n136.1205 42.94\r\n137.0964 83.33\r\n143.0857 74.74\r\n145.1012 243.07\r\n147.1171 377.63\r\n148.1206 33.49\r\n148.3084 34.28\r\n149.0961 80.16\r\n149.1327 238.82\r\n150.1358 34.85\r\n151.1118 70.27\r\n155.0855 64.54\r\n157.1014 88.33\r\n158.0815 332.28\r\n159.1171 276.12\r\n161.13

In [5]:
import pandas as pd
from massql import msql_engine

ALL_MASSQL_QUERIES = {

    # monohydroxy
    'Monohydroxy_stage1': "QUERY scaninfo(MS2DATA) WHERE MS2PROD=341.28:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PROD=323.27:TOLERANCEMZ=0.01:INTENSITYPERCENT=2",
    'Monohydroxy_stage2' : "QUERY scaninfo(MS2DATA) WHERE MS2PROD=341.28:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PROD=323.27:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PREC=X AND MS2PROD=X-358.2871:TOLERANCEMZ=0.01:INTENSITYPERCENT=2",
    'Mono-3a-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=107.086:TOLERANCEPPM=20:INTENSITYPERCENT=10:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=109.101:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.8:INTENSITYMATCHPERCENT=20",
    'Mono-7b-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=177.127:TOLERANCEPPM=20:INTENSITYPERCENT=10:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=241.194:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.2:INTENSITYMATCHPERCENT=30",

    # dihydroxy
    'Dihydroxy_stage1': "QUERY scaninfo(MS2DATA) WHERE MS2PROD=339.27:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PROD=321.26:TOLERANCEMZ=0.01:INTENSITYPERCENT=2",
    'Dihydroxy_stage2': "QUERY scaninfo(MS2DATA) WHERE MS2PROD=339.27:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PROD=321.26:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PREC=X AND MS2PROD=X-374.2894:TOLERANCEMZ=0.01:INTENSITYPERCENT=2",
    'Di-1-ketone': "QUERY scaninfo(MS2DATA) WHERE MS2PROD=161.132:TOLERANCEPPM=20:INTENSITYPERCENT=95",
    'Di-3,12a-OH; 7,12a-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=211.147:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=201.163:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.6:INTENSITYMATCHPERCENT=80",
    'Di-3,12a-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=243.174:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=239.179:TOLERANCEPPM=20:INTENSITYMATCH=Y*1:INTENSITYMATCHPERCENT=30",
    'Di-7,12a-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=243.174:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=239.179:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.25:INTENSITYMATCHPERCENT=30",
    'Di-3,7-OH; 3,6-OH; 3,12b-OH; 7,12b-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=211.147:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=201.163:TOLERANCEPPM=20:INTENSITYMATCH=Y*4.2:INTENSITYMATCHPERCENT=40",
    'Di-3,7-OH; 3,12b-OH; 7,12b-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=161.132:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=201.163:TOLERANCEPPM=20:INTENSITYMATCH=Y*2:INTENSITYMATCHPERCENT=20",
    'Di-3,6-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=161.132:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=201.163:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.7:INTENSITYMATCHPERCENT=60",
    'Di-3,7-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=175.148:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=173.132:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.5:INTENSITYMATCHPERCENT=30",
    'Di-3,12b-OH; 7,12b-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=175.148:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=173.132:TOLERANCEPPM=20:INTENSITYMATCH=Y*1:INTENSITYMATCHPERCENT=20",

    # trihydroxy
    'Trihydroxy_stage1': "QUERY scaninfo(MS2DATA) WHERE MS2PROD=337.25:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PROD=319.24:TOLERANCEMZ=0.01:INTENSITYPERCENT=2",
    'Trihydroxy_stage2': "QUERY scaninfo(MS2DATA) WHERE MS2PROD=337.25:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PROD=319.24:TOLERANCEMZ=0.01:INTENSITYPERCENT=2 AND MS2PREC=X AND MS2PROD=X-390.277:TOLERANCEMZ=0.01:INTENSITYPERCENT=2",
    'All_ketone_NEW': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=107.086:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=105.07:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.55:INTENSITYMATCHPERCENT=40 AND MS2MZ=199.148:TOLERANCEPPM=20:INTENSITYMATCH=X:INTENSITYMATCHREFERENCE AND MS2MZ=209.132:TOLERANCEPPM=20:INTENSITYMATCH=X*0.6:INTENSITYMATCHPERCENT=45",
    '3k_7or12_OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=199.148:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=213.163:TOLERANCEPPM=20:INTENSITYMATCH=Y*2:INTENSITYMATCHPERCENT=20",
    '3a_12k': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=319.242:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=309.257:TOLERANCEPPM=20:INTENSITYMATCH=Y*5.5:INTENSITYMATCHPERCENT=20",
    '3,6,7_NEW': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=191.143:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=225.163:TOLERANCEPPM=20:INTENSITYMATCH=Y*1.2:INTENSITYMATCHPERCENT=60 AND MS2MZ=199.148:TOLERANCEPPM=20:INTENSITYMATCH=X:INTENSITYMATCHREFERENCE AND MS2MZ=213.163:TOLERANCEPPM=20:INTENSITYMATCH=X*0.75:INTENSITYMATCHPERCENT=20",
    '3,7a,12a-OH_NEW': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=209.132:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=199.148:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.6:INTENSITYMATCHPERCENT=45",
    '3,6a,7a-OH' : "QUERY scaninfo(MS2DATA) WHERE MS2MZ=173.132:TOLERANCEPPM=20:INTENSITYMATCH=X:INTENSITYMATCHREFERENCE AND MS2MZ=185.132:TOLERANCEPPM=20:INTENSITYMATCH=X*1:INTENSITYMATCHPERCENT=5",
    '3,6b,7a-OH_3,6a,7b-OH': "QUERY scaninfo(MS2DATA) WHERE MS2MZ=159.117:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=161.132:TOLERANCEPPM=20:INTENSITYMATCH=Y*0.9:INTENSITYMATCHPERCENT=10",
    '3,6b,7a-OH' : "QUERY scaninfo(MS2DATA) WHERE MS2MZ=125.096:TOLERANCEPPM=20:INTENSITYMATCH=Y:INTENSITYMATCHREFERENCE AND MS2MZ=309.257:TOLERANCEPPM=20:INTENSITYMATCH=Y*1:INTENSITYMATCHPERCENT=20"
}

def massql_filter(input_mgf, massql_queries=ALL_MASSQL_QUERIES):

    out_df = []

    for query_name, input_query in massql_queries.items():
        print(f"Running query: {query_name}")
        results_df = msql_engine.process_query(input_query, input_mgf)
        if len(results_df) == 0:
            out_df.append({'query': query_name, 'scan_list': 'NA'})
            continue
        passed_scan_ls = results_df['scan'].values.tolist()
        passed_scan_ls = [int(x) for x in passed_scan_ls]
        out_df.append({'query': query_name, 'scan_list': passed_scan_ls})

    out_name = input_mgf.replace('.mgf', '_massql_ALL_NEW.tsv')
    out_df = pd.DataFrame(out_df)
    out_df.to_csv(out_name, sep='\t', index=False)
    print(f"Saved results to {out_name}")


In [6]:
input_mgf = 'Attached standard file_ GCDCA_TCA_for_authors_to_reproduce_GitHub_error.mgf'  # must match uploaded filename
massql_filter(input_mgf=input_mgf, massql_queries=ALL_MASSQL_QUERIES)


Running query: Monohydroxy_stage1
TOTAL QUERIES 1


100%|██████████| 1/1 [00:00<00:00, 70.81it/s]


Running query: Monohydroxy_stage2


0it [00:00, ?it/s]


TOTAL QUERIES 0


...

100%|██████████| 1/1 [00:00<00:00, 57.98it/s]

Running query: 3,6b,7a-OH


TOTAL QUERIES 1


100%|██████████| 1/1 [00:00<00:00, 138.83it/s]

Saved results to Attached standard file_ GCDCA_TCA_for_authors_to_reproduce_GitHub_error_massql_ALL_NEW.tsv


In [8]:
from google.colab import files

files.download('Attached standard file_ GCDCA_TCA_for_authors_to_reproduce_GitHub_error_massql_ALL_NEW.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>